# Adding lightcurve effects to LSST Solar System Simulations
By Jamie Robinson

LSST SSSC Sprint June 2022

This notebook demonstrates the work I did at the Sprint. I wanted to add changes in asteroid brightness due to rotation and also the longer term changes of aspect angle that leads to shifts in brightness (on top of the already modelled distance and phase effects).

The asteroid is modelled as a triaxial ellipsoid rotating around its shortest axis
- axes a > b > c
- spin pole orientation (ecliptic longitude and latitude)
- rotation period and simple sinusoidal lightcurve

This shape model is applied to some asteroid from the simulated database. The database contains only sparse measurements as expected from the LSST cadence. Therefore we take the orbital elements and propagate using [sbpy open_orb](https://sbpy.readthedocs.io/en/latest/sbpy/data/orbit.html) to densely sample the asteroid positional coordinates.

These positions and and shape parameters are fed into the equations from [Fernández-Valenzuela 2022](https://arxiv.org/abs/2202.02374v1) and so a shift from the expected brightness due to lightcurve effects can be calculated for any observation made with LSST.

Based on the demo notebook by Juric et al., available from https://github.com/lsst-sssc/lsst-simulation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from astropy import units as u
from astropy.coordinates import SkyCoord, GCRS
from sbpy.data import Orbit
from astropy.time import Time
from astropy.coordinates import solar_system_ephemeris
from sbpy.photometry import HG
from astropy.table import QTable

In [ ]:
from lsst.rsp import get_tap_service

service = get_tap_service("ssotap")
assert service is not None

In [ ]:
# ssObjectId of object to analyse
ssoid = "6098332225018"  # good test object

In [ ]:
query = """
SELECT
    *
FROM
    dp03_catalogs_10yr.DiaSource as dia
INNER JOIN
    dp03_catalogs_10yr.SSSource as sss
ON
    dia.diaSourceId = sss.diaSourceId
WHERE
    dia.ssObjectId={}
ORDER by dia.ssObjectId
""".format(
    ssoid
)

df = service.search(query).to_table().to_pandas()
df

In [ ]:
# calculate elongation angle
R = df["heliocentricDist"]
Delta = df["topocentricDist"]
alpha = np.radians(df["phaseAngle"])

R_E = np.sqrt((R * R) + (Delta * Delta) - (2.0 * R * Delta * np.cos(alpha)))
df["elong"] = np.degrees(np.arccos(((R_E * R_E) + (Delta * Delta) - (R * R)) / (2.0 * R_E * Delta)))

In [ ]:
# calculate reduced mag
df["reduced_mag"] = df["mag"] - 5 * np.log10(df["topocentricDist"] * df["heliocentricDist"])

In [ ]:
# put obs in time order
df = df.sort_values("midPointMjdTai")

In [ ]:
# get info on the object

In [ ]:
results = service.search("SELECT * FROM dp03_catalogs_10yr.SSObject " "WHERE ssObjectId={}".format(ssoid))
df_obj = results.to_table().to_pandas()

In [ ]:
df_obj

In [ ]:
# get orbit

In [ ]:
results = service.search("SELECT * FROM dp03_catalogs_10yr.MPCORB " "WHERE ssObjectId={}".format(ssoid))
df_orb = results.to_table().to_pandas()

In [ ]:
df_orb

### Resample the sparse observations by propagating orbit with sbpy & oorb

In [ ]:
# check orbit
e = df_orb["e"]
incl = df_orb["incl"]
q = df_orb["q"]
a = q / (1.0 - e)
Q = a * (1.0 + e)
print(a, e, incl)

In [ ]:
df_orb["a"] = a
df_orb["Q"] = Q
df_orb

In [ ]:
list(df_orb)

In [ ]:
# Calculate some extra orbital elements
df_orb["P"] = df_orb["a"] ** (3.0 / 2.0)  # orbital period in years
df_orb["n"] = 360.0 / (df_orb["P"] * 365.25)  # mean motion in deg/day
df_orb["M"] = (
    df_orb["n"] * (df_orb["epoch"] - df_orb["tperi"])
) % 360  # angles must be in correct range otherwise sbpy/pyoorb freak out
df_orb[["a", "P", "n", "M"]]

In [ ]:
# rename columns for consistency
df_orb = df_orb.rename(columns={"node": "Omega", "peri": "w"})

In [ ]:
df_orb[["a", "e", "incl", "Omega", "w", "M"]]

In [ ]:
# create an sbpy oorb object from dataframe via QTable
tab = QTable.from_pandas(
    df_orb[["a", "e", "incl", "Omega", "w", "M"]],
    units={"a": u.au, "incl": u.deg, "Omega": u.deg, "w": u.deg, "M": u.deg},
)
orbit = Orbit.from_table(tab)

In [ ]:
# oorb requires certain extra fields
orbit["epoch"] = Time(Time(df_orb["epoch"], format="mjd").jd, format="jd")
orbit["targetname"] = np.array(df_orb["ssObjectId"]).astype(str)
orbit["H"] = df_orb["mpcH"] * u.mag
orbit["G"] = df_orb["mpcG"] * u.dimensionless_unscaled

In [ ]:
orbit

In [ ]:
# define a set of JD times to propagate the orbital elements to
N = 1000
# N=10000
times = Time(
    Time(np.linspace(np.amin(df["midPointMjdTai"]), np.amax(df["midPointMjdTai"]), N), format="mjd").jd,
    format="jd",
)
times[0]

In [ ]:
# create an empty dataframe to hold resampled observations
df_dense = pd.DataFrame()
df_dense["midPointMjdTai"] = times.mjd

In [ ]:
# propagate the orbit forward in time.
# probably a better way to do this but I can't get oo_propagate to work with a Time list right now
# see: https://github.com/NASA-Planetary-Science/sbpy/issues/341
# dependencies:
# conda install conda-forge::openorb
# pip install jplephem

df_pos = pd.DataFrame()  # empty dataframe to hold cartesian coordinates

for i in range(len(times)):
    print(i)
    prop_elem = orbit.oo_propagate(times[i])  # propagate the orbit to the selected time step
    del prop_elem.table[
        "orbtype"
    ]  # orbtype is added as int, sbpy freaks out so delete the orbtype and then _to_oo works it out
    print("propagate")
    statevec = prop_elem.oo_transform("CART")  # transform from orbital elements to cartesian
    print("transform")

    # append new cartesian coordinates to the dataframe
    _df_statevec = statevec.table.to_pandas()
    df_pos = pd.concat((df_pos, _df_statevec))

df_pos.reset_index(drop=True, inplace=True)

In [ ]:
df_pos

### Use astropy coordinates to transform between all the coordinate systems

In [ ]:
# define heliocentric cartesian coordinates
c_xyz_hel = SkyCoord(
    x=np.array(df_pos["x"]),
    y=np.array(df_pos["y"]),
    z=np.array(df_pos["z"]),
    unit="AU",
    representation_type="cartesian",
    frame="heliocentrictrueecliptic",
)

In [ ]:
# transform to heliocentric ecliptic coords
c_ecl_hel = c_xyz_hel.copy()
c_ecl_hel.representation_type = "spherical"

In [ ]:
# transform to geocentric equatorial coords (times required to calculate Earth position)
with solar_system_ephemeris.set("jpl"):
    c_eq_geo = c_xyz_hel.transform_to(GCRS(obstime=times))

In [ ]:
# transform to geocentric cartesian coords
c_xyz_geo = c_eq_geo.copy()
c_xyz_geo.representation_type = "cartesian"

In [ ]:
# transform from geo equatorial (ra, dec) to geo ecliptic (lon, lat)
c_ecl_geo = c_eq_geo.transform_to("geocentrictrueecliptic")

In [ ]:
# plot the propagated cartesian positions against the database values

x_plot = "midPointMjdTai"
y_plot1 = "heliocentricX"
y_plot2 = "heliocentricY"
y_plot3 = "heliocentricZ"
df_plot = df

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

x = ax1.scatter(df_plot[x_plot], df_plot[y_plot1], label=y_plot1)
x = ax1.scatter(df_plot[x_plot], df_plot[y_plot2], label=y_plot2)
x = ax1.scatter(df_plot[x_plot], df_plot[y_plot3], label=y_plot3)

ax1.plot(times.mjd, c_xyz_hel.x)
ax1.plot(times.mjd, c_xyz_hel.y)
ax1.plot(times.mjd, c_xyz_hel.z)

ax1.set_xlabel(x_plot)
ax1.set_ylabel("distance")
ax1.legend()

plt.show()

There are some deviations of x and y positions, probably due to slightly different reference frames and methods of propagating orbits.

There is something wrong with database z positions, will be fixed soon!

In [ ]:
# the ecliptic coordinates look good!

x_plot = "midPointMjdTai"
y_plot1 = "eclipticLambda"
y_plot2 = "eclipticBeta"
df_plot = df

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

x = ax1.scatter(df_plot[x_plot], df_plot[y_plot1], label=y_plot1)
x = ax1.scatter(df_plot[x_plot], df_plot[y_plot2], label=y_plot2)

ax1.plot(times.mjd, c_ecl_geo.lon.degree)
ax1.plot(times.mjd, c_ecl_geo.lat.degree)

ax1.set_xlabel(x_plot)
ax1.set_ylabel("angle")
ax1.legend()

plt.show()

### Define some physical properties of our asteroid shape and rotation model

In [ ]:
# calculate the spherical radius of the asteroid, given its absolute magnitude from the simulated database
C = 1330  # km
albedo = 0.15  # pick a typical asteroid albedo
abs_mag = df_obj.iloc[0]["r_H"]
radius = 0.5 * ((C / np.sqrt(albedo)) * (10 ** (-0.2 * abs_mag)))
print("radius = {} km".format(radius))

In [ ]:
print("circular area = {} km^2".format(np.pi * radius * radius))

In [ ]:
volume = (4.0 / 3.0) * np.pi * (radius**3.0)
print("volume = {} km^3".format(volume))

In [ ]:
def ellipsoid_axes(V, b_a, c_a):
    """
    Find the axes of an ellipsoid given a volume V and axes ratios.
    b_a is the b/a ratio
    c_a is the c/a ratio
    """
    a = (((3.0 * V) / (4.0 * np.pi)) * (1.0 / b_a) * (1.0 / c_a)) ** (1.0 / 3.0)
    b = b_a * a
    c = c_a * a
    return a, b, c

In [ ]:
# define the triaxial ellipsoid axes
a, b, c = ellipsoid_axes(volume, 0.75, 0.5)
print(a, b, c)
print("volume = {} km^3".format((4.0 / 3.0) * np.pi * a * b * c))

In [ ]:
# set rotational properties
period = 5.0 / 24  # rotation period (days)
# period = 53/24 # rotation period (days)
t0 = 0  # reference time for lightcurve phase
lambda_pole = 0  # spin pole ecliptic longitude
beta_pole = np.pi / 2.0  # spin pole ecliptic latitude

### Calculate asteroid aspect angle from ecliptic coordinates

In [ ]:
def aspect_angle(lambda_obj, beta_obj, lambda_pole=0, beta_pole=np.pi / 2.0):
    """
    Calculate the aspect angle as a function of ecliptic coords of object and pole (radians)
    (Fernandez-Valenzuela et al. 2022)
    """
    return (np.pi / 2.0) - np.sin(
        (np.sin(beta_obj) * np.sin(beta_pole))
        + (np.cos(beta_obj) * np.cos(beta_pole) * np.cos(lambda_obj - lambda_pole))
    )

In [ ]:
# calculate aspect angle for each observation
df["aspect(deg)"] = np.degrees(
    aspect_angle(np.radians(df["eclipticLambda"]), np.radians(df["eclipticBeta"]), lambda_pole, beta_pole)
)

In [ ]:
# find the aspect angle but for the more dense & uniform sampled times
df_dense["aspect(deg)"] = np.degrees(aspect_angle(c_ecl_geo.lon.radian, c_ecl_geo.lat.radian))

In [ ]:
x_plot = "midPointMjdTai"
y_plot = "aspect(deg)"
c_plot = "phaseAngle"
df_plot = df
df_plot2 = df_dense

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

s1 = ax1.scatter(df_plot[x_plot], df_plot[y_plot], c=df_plot[c_plot])
cbar1 = fig.colorbar(s1)
cbar1.set_label(c_plot)

ax1.plot(df_plot2[x_plot], df_plot2[y_plot])

ax1.set_xlabel(x_plot)
ax1.set_ylabel(y_plot)

plt.show()

### Find the lightcurve amplitude as a function of aspect angle and asteroid shape

In [ ]:
def lightcurve_amplitude(aspect_angle, a=1, b=0.75, c=0.5):
    """
    Calculate the lightcurve amplitude for a trixial ellipsoid of axis ratio a:b:c, as a function of aspect angle
    """
    X = 2.5 * np.log10(a / b)
    Y = ((a * np.cos(aspect_angle)) ** 2.0) + ((c * np.sin(aspect_angle)) ** 2.0)
    Z = ((b * np.cos(aspect_angle)) ** 2.0) + ((c * np.sin(aspect_angle)) ** 2.0)
    return X - (1.25 * np.log10(Y / Z))

In [ ]:
lc_amp = lightcurve_amplitude(df["aspect(deg)"])
df["lc_amp(mag)"] = lc_amp

In [ ]:
df_dense["lc_amp(mag)"] = lightcurve_amplitude(df_dense["aspect(deg)"])

In [ ]:
x_plot = "midPointMjdTai"
y_plot = "lc_amp(mag)"
c_plot = "aspect(deg)"
df_plot = df
df_plot2 = df_dense

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

s1 = ax1.scatter(df_plot[x_plot], df_plot[y_plot], c=df_plot[c_plot])
cbar1 = fig.colorbar(s1)
cbar1.set_label(c_plot)

ax1.plot(df_plot2[x_plot], df_plot2[y_plot])

ax1.set_xlabel(x_plot)
ax1.set_ylabel(y_plot)

plt.show()

### find the projected asteroid area as a function of aspect angle and shape

In [ ]:
def area_aspect(aspect_angle, a=1, b=0.75, c=0.5):
    """
    Calculate the projected areas of a triaxial ellipsoid as a function of aspect angle.
    The axes a,b,c should be in appropriate units of length to calculate absolute magnitude
    """
    area_min = np.pi * b * np.sqrt(((a * np.cos(aspect_angle)) ** 2.0) + ((c * np.sin(aspect_angle)) ** 2.0))
    area_max = np.pi * a * np.sqrt(((b * np.cos(aspect_angle)) ** 2.0) + ((c * np.sin(aspect_angle)) ** 2.0))
    area_avg = (area_max + area_min) / 2.0
    return area_avg

In [ ]:
# find the projected area of asteroid as a function of time
df["projected_area(km^2)"] = area_aspect(df["aspect(deg)"], a, b, c)

In [ ]:
df_dense["projected_area(km^2)"] = area_aspect(df_dense["aspect(deg)"], a, b, c)

In [ ]:
x_plot = "midPointMjdTai"
y_plot = "projected_area(km^2)"
c_plot = "aspect(deg)"
df_plot = df
df_plot2 = df_dense

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

s1 = ax1.scatter(df_plot[x_plot], df_plot[y_plot], c=df_plot[c_plot])
cbar1 = fig.colorbar(s1)
cbar1.set_label(c_plot)

ax1.plot(df_plot2[x_plot], df_plot2[y_plot])

ax1.set_xlabel(x_plot)
ax1.set_ylabel(y_plot)

plt.show()

### find the change in absolute magnitude as projected area changes

In [ ]:
delta_H = 2.5 * np.log10((np.pi * radius * radius) / (df["projected_area(km^2)"]))
df["delta_H(mag)"] = delta_H

In [ ]:
df_dense["delta_H(mag)"] = 2.5 * np.log10((np.pi * radius * radius) / (df_dense["projected_area(km^2)"]))

In [ ]:
x_plot = "midPointMjdTai"
y_plot = "delta_H(mag)"
c_plot = "aspect(deg)"
df_plot = df

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

s1 = ax1.scatter(df_plot[x_plot], df_plot[y_plot], c=df_plot[c_plot])
cbar1 = fig.colorbar(s1)
cbar1.set_label(c_plot)

ax1.plot(df_plot2[x_plot], df_plot2[y_plot])

ax1.set_xlabel(x_plot)
ax1.set_ylabel(y_plot)

plt.show()

### Calculate the rotational brightness variation

In [ ]:
m_rot = (df["lc_amp(mag)"] / 2.0) * np.sin(
    (2 * np.pi * (df["midPointMjdTai"] - t0)) / period
)  # assume simple sinusoidal lightcurve
df["m_rot(mag)"] = m_rot

In [ ]:
df_dense["m_rot(mag)"] = (df_dense["lc_amp(mag)"] / 2.0) * np.sin(
    (2 * np.pi * (df_dense["midPointMjdTai"] - t0)) / period
)

In [ ]:
df

In [ ]:
df_dense

In [ ]:
np.amax(df_dense["lc_amp(mag)"])

### Add all rotation and shape effects to the observed brightness

In [ ]:
# the lightcurve effects leads to the apparition effect, distinct shifts in brightness with the seasons

x_plot = "phaseAngle"
y_plot = "reduced_mag"
y_plot1 = "delta_H(mag)"
y_plot2 = "m_rot(mag)"
df_plot = df[df["band"] == "r"]

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

x = ax1.scatter(df_plot[x_plot], df_plot[y_plot], label="no lightcurve")
x = ax1.scatter(df_plot[x_plot], df_plot[y_plot] + df_plot[y_plot1] + df_plot[y_plot2], label="lightcurve")

ax1.set_xlabel(x_plot)
ax1.set_ylabel(y_plot)
ax1.invert_yaxis()
ax1.legend()

plt.show()

In [ ]:
x_plot = "phaseAngle"
y_plot = "reduced_mag"
y_plot1 = "delta_H(mag)"
y_plot2 = "m_rot(mag)"
df_plot = df[df["band"] == "r"]

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

x = ax1.scatter(
    df_plot[x_plot],
    df_plot[y_plot] + df_plot[y_plot1] + df_plot[y_plot2],
    c=df_plot["midPointMjdTai"],
    label="lightcurve",
)

ax1.set_xlabel(x_plot)
ax1.set_ylabel(y_plot)
ax1.invert_yaxis()
ax1.legend()

plt.show()

In [ ]:
x_plot = "midPointMjdTai"
y_plot = "reduced_mag"
y_plot1 = "delta_H(mag)"
y_plot2 = "m_rot(mag)"
df_plot = df[df["band"] == "r"]

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

x = ax1.scatter(df_plot[x_plot], df_plot[y_plot], label="no lightcurve")
x = ax1.scatter(df_plot[x_plot], df_plot[y_plot] + df_plot[y_plot1] + df_plot[y_plot2], label="lightcurve")

ax1.set_xlabel(x_plot)
ax1.set_ylabel(y_plot)
ax1.invert_yaxis()
ax1.legend()

plt.show()

In [ ]:
x_plot = "midPointMjdTai"
y_plot1 = "delta_H(mag)"
y_plot2 = "m_rot(mag)"
c_plot = "aspect(deg)"
df_plot = df[df["band"] == "r"]

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

s1 = ax1.scatter(df_plot[x_plot], df_plot[y_plot1] + df_plot[y_plot2], c=df_plot[c_plot], zorder=1)
cbar1 = fig.colorbar(s1)
cbar1.set_label(c_plot)

ax1.plot(times.mjd, df_plot2[y_plot1] + df_plot2[y_plot2], zorder=0)

ax1.set_xlabel(x_plot)
ax1.set_ylabel("delta_H + m_rot")
ax1.invert_yaxis()

plt.show()

There are some deviations of the shifted database brightnesses from our model which we have propagated ourselves. This is probably something to do with the problems with z-components as mentioned above.